### Figure 3 | Transcriptome-based models discriminate sarcoidosis more accurately than demographic features
Comparison of cross-validated classification performance using transcriptomic features, principal components, and demographic variables.